In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os.path
import os
from scipy.interpolate import interp1d
from scipy.signal import butter,filtfilt
from scipy.optimize import curve_fit
from scipy import odr
import scipy.linalg as la

#%matplotlib inline
%matplotlib qt
%run Functions_for_RUS_LabView.ipynb

In [42]:
folder = "C:\\Users\\j111\\Box Sync\\Projects"
folder = "C:\\Users\\Florian\\Box Sync\\Projects"
project = "\\Mn3Ge\\RUS\\Mn3Ge_2007A\\temp_dependent_data_with_Labview\\improved_setup\\good_data"
# project = "\\Mn3.019Sn0.981\\RUS\\2007A\\good_data"
project = "/UTe2/RUS/UTe2_2007C/PLL/good data"

data_files = os.listdir(folder+project)
data_files = [i for i in data_files if i[-4:]=='.dat']
np.array(data_files)

array(['1677kHz_up_clean.dat', '1894kHz_up_clean.dat',
       '2003kHz_up_clean.dat', '2234kHz_up_clean.dat',
       '2367kHz_down_clean.dat', '2522kHz_up_clean.dat',
       '2641kHz_up_clean.dat', '2750kHz_down.dat', '2791kHz_down.dat',
       '3397kHz_up_clean.dat'], dtype='<U22')

In [43]:
filenames = [folder + project + '\\' + i for i in data_files[:]]
Traw = []
fraw = []
gammaraw = []

for name in filenames:
    individual_data = []
    T, f= import_data(name, 1, 2)    
    Traw.append(np.array(T))
    fraw.append(np.array(f))
    #gammaraw.append(np.array(gamma))
    
# write data into seperate arrays for temperature "T" and frequency "f"
Traw = np.array(Traw) # temperature in K
fraw = np.array(fraw) # frequencies in kHz
#gammaraw = np.array(gammaraw) # resonance widths in kHz

In [46]:
Tmin = 1.274
Tmax = 1.7
# Tmin = 355
# Tmax = 385
mask = np.array([(i > Tmin) & (i < Tmax) for i in Traw])
labels = ([str(np.round(i[-1]/1e3, 0))[:-2]+' kHz' for i in fraw])

plt.figure(figsize=(9.5,7))
palette = plt.get_cmap('viridis')
for i in np.arange(len(Traw)):
   # plt.scatter(Traw[i][mask[i]], ((fraw[i]-fraw[i][-1])/fraw[i][-1]*1e5)[mask[i]], s=10, color=palette(i/(len(Traw)-1)), label=labels[i])
   plt.scatter(Traw[i][mask[i]], fraw[i][mask[i]], s=10, color=palette(i/(len(Traw)-1)), label=labels[i])


plt.xlabel('T (K)',fontsize=22)
plt.ylabel('$\\mathrm{\\Delta f / f}$ $(10^{-5})$',fontsize=22)
plt.xlim(Tmin-0.01, Tmax+0.01)
plt.legend(fontsize=18, markerscale=4)
plt.tick_params(axis="both",direction="in", labelsize=20, bottom='True', top='True', left='True', right='True', 
                length=4, width=1, which = 'major')

plt.show()

# Interpolate the data

In [27]:
# this creates interpolated frequency data at the same temperature points in the biggest temperature
# range possible (all frequency curves have to be measured in at least this temperature range)
Tmin_int = max([min(t) for t in Traw])
Tmax_int = min([max(t) for t in Traw])
# Tmin_int = 365
# Tmax_int = 375
n = np.mean(np.array([len(t) for t in Traw]))
Tint = np.linspace(Tmin_int, Tmax_int, int(n))

fint = []
gint = []
# for i in np.arange(len(Traw)):
#     fi, gi = interpolate(Traw[i], fraw[i], gammaraw[i], Tint, 'linear')
#     fint.append(fi)
#     gint.append(gi)

for i in np.arange(len(Traw)):
    fint.append( interp1d(Traw[i], fraw[i], kind='linear')(Tint) )

fint = np.array(fint)
dfint = np.array([(i-i[-1]) for i in fint])
dfofint = np.array([(i-i[-1]/i) for i in fint])
#gint = np.array(gint)


# mask the raw data arrays such that df is zero at the same temperature
mask_raw = np.array([(i >= Tmin_int) & (i <= Tmax_int) for i in Traw])
Traw = np.array([Traw[i][mask_raw[i]] for i in np.arange(len(Traw))])
fraw = np.array([fraw[i][mask_raw[i]] for i in np.arange(len(Traw))])
dfraw = np.array([(i-i[-1]) for i in fraw])
dfofraw = np.array([(i-i[-1])/i[-1] for i in fraw])
#graw = np.array([gammaraw[i][mask_raw[i]] for i in np.arange(len(Traw))])

print('the maximum temperature range is '+str(Tmin_int)+' to '+str(Tmax_int)+' K')

Tlp = Tint
flp = fint
dflp = dfint
dfoflp = dfofint

the maximum temperature range is 365.0443 to 374.9553 K


# Plot the interpolated curves

In [28]:
# this just plots the filtered frequency over the raw data

Tmin = 1
Tmax = 2
Tmin = 355
Tmax = 385

# which frequencies do you want to plot
resonances = np.arange(len(flp))
#resonances = [0,5,10,12]


plt.figure(figsize=(10,7))

# make scatter plot of raw data
mask_raw = np.array([(i >= Tmin) & (i <= Tmax) for i in Traw])

#shift = [.3, 0, .4, -0.3, -1]
#index = [1, 3, 5, 9, 13]
#shift = 7
for i in resonances:
    palette = plt.get_cmap('brg')
    plt.scatter(Traw[i][mask_raw[i]], dfraw[i][mask_raw[i]], color=palette(i/(len(flp)-1)), s=10)
    

# make plot of interpolation
mask_lp = (Tint >= Tmin) & (Tint <= Tmax)

shift = 7
for i in resonances:
    palette = plt.get_cmap('Set1')
    plt.plot(Tlp[mask_lp], dflp[i][mask_lp], color=palette(i/len(flp)), label=(str(round(fraw[i][-1], 1))+', '+str(i)))


plt.xlabel('T (K)',fontsize=20)
plt.ylabel('$\mathrm{\Delta f\,\,(kHz)}$',fontsize=20)
plt.tick_params(axis="both",direction="in", labelsize=18, bottom='True', top='True', left='True', right='True', length=4, width=1, which = 'major')

#plt.ylim([-40, 20])
#plt.xlim(360, 380)

plt.legend(loc=(.6, .04), fontsize=17)

# get principal components

In [29]:
fnew = dflp[:,::10]
# fmean = np.array([i-np.mean(i) for i in fnew])
# fnew = fmean
print(len(fnew))
Tpv = Tlp[::10]
M = np.dot( np.transpose(fnew), fnew )

print(M.shape)

14
(41, 41)


In [30]:
val, vec = np.linalg.eigh(M)
print(vec)
print(np.max(vec))
#val = val[::-1]
#vec = vec[::-1]

[[ 6.09978005e-01 -1.65355444e-01 -1.82065418e-01 ... -1.21336016e-01
  -3.07094519e-01  2.61253129e-01]
 [-6.89071492e-01 -1.31050433e-01 -8.64450443e-02 ... -9.76530830e-02
  -2.80540950e-01  2.58864242e-01]
 [ 2.98860210e-02  3.01842345e-01  1.80677433e-02 ... -7.30871522e-02
  -2.50426637e-01  2.56407012e-01]
 ...
 [-6.03471058e-02  4.18515210e-02 -2.53354986e-02 ... -5.27996337e-02
  -8.92975043e-04  3.77434045e-03]
 [-3.19405778e-02  2.53789327e-02 -1.61236090e-01 ... -3.68481277e-02
  -9.30692031e-05  2.21526542e-03]
 [-2.82421472e-03  4.93536130e-03 -5.80793764e-02 ... -8.15710507e-03
   6.57295873e-05  5.96067508e-04]]
0.7492106805653009


In [36]:
plt.figure(figsize=(10,7))
mask = (Tpv>1.29) & (Tpv < 1.65)
mask = (Tpv>355) & (Tpv<385)

# plt.plot(Tpv[mask], -vec[:,-1][mask]+0.06, label=str(round(val[-1], 0))[:-2], linewidth=4 )
# plt.plot(Tpv[mask], -vec[:,-2][mask]*1.2+0.01, label=str(round(val[-2], 0))[:-2], linewidth=4 )
# plt.plot(Tpv[mask], vec[:,-3][mask]/1.7, label=str(round(val[-3], 0))[:-2], linewidth=4 )
plt.plot(Tpv[mask], -vec[:,-1][mask], label=str(round(val[-1], 0))[:-2], linewidth=4 )
plt.plot(Tpv[mask], -vec[:,-2][mask], label=str(round(val[-2], 0))[:-2], linewidth=4 )
plt.plot(Tpv[mask], -vec[:,-3][mask], label=str(round(val[-3], 0))[:-2], linewidth=4 )
# plt.plot(Tpv[mask], vec[:,-4][mask]/1.7, label=str(round(val[-4], 0))[:-2], linewidth=4 )
# plt.plot(Tpv[mask], vec[:,-5][mask]/1.7, label=str(round(val[-5], 0))[:-2], linewidth=4 )

#plt.plot(Tpv, -vec[:,-4]/1.5-0.02, label=str(round(val[-4], 0))[:-2], linewidth=4 )

plt.xlabel('T (K)',fontsize=20)
#plt.ylabel('$\\phi_\\mathrm{i}$ (a.u.)',fontsize=20)
plt.ylabel('Eigenvectors (a.u.)',fontsize=20)
plt.tick_params(axis="x",direction="in", labelsize=18, bottom='True', top='True', left='True', right='True', length=4, width=1, which = 'major')
plt.tick_params(axis="y",direction="in", labelsize=0, bottom='True', top='True', left='False', right='False', length=0, width=0, which = 'major')


legend = plt.legend(title='Eigenvalues', fontsize=18)
legend.get_title().set_fontsize('18')
# plt.xlim(1.285, 1.655)

In [14]:
val[-10:]

array([5.58214311e-01, 8.89959766e-01, 2.28487001e+00, 5.42515877e+00,
       1.79052640e+01, 2.52744051e+01, 7.24919560e+01, 7.43854233e+03,
       3.25439549e+04, 4.41032923e+06])

# get projection of frequencies on principal components

In [70]:
fnew.shape

(10, 1869)

In [190]:
N = 3
vec[:,-1] = -vec[:,-1]
vec[:,-2] = -vec[:,-2]
pc = np.transpose(vec)[-N::][::-1]
pv = val[-N::][::-1]

projection = np.dot(fnew, np.transpose(pc))
fcalc = np.array( [ sum(np.array([projection[i,k]*pc[k] for k in  np.arange(len(pc))])) for i in np.arange(len(projection))] )

In [191]:
row_sum = np.transpose( np.array( list([ sum(row) for row in projection]) * projection.shape[1] ).reshape(projection.shape[::-1]) )
percentage = projection/row_sum
percentage

array([[ 0.83495666,  0.06444477,  0.10059857],
       [ 0.98670544, -0.02501142,  0.03830598],
       [ 1.01804715,  0.05195195, -0.0699991 ],
       [ 0.42105319,  0.48882667,  0.09012014],
       [ 1.1496876 , -0.0044412 , -0.1452464 ],
       [ 0.61239861, -0.00215793,  0.38975933],
       [ 1.05873344,  0.17130866, -0.23004209],
       [ 1.06241861, -0.13994468,  0.07752607],
       [ 0.51213693,  0.24516273,  0.24270034],
       [ 1.02418541, -0.01940292, -0.00478249]])

In [192]:
plt.figure(figsize=(10,8))
mask = (Tpv>1.29) & (Tpv<1.65)

palette = plt.get_cmap('viridis')

shift = [4, 2.6, 1.2, -0.2]
k=0
for i in [1, 5, 6, 8]:
    plt.plot(Tpv[mask], fnew[i][mask]/max(fnew[i][mask]) +shift[k], label=(str(np.round(fraw[i][-1]/1e3, 0))[:-2]+' kHz') , color=palette((k)/(3)) , linewidth=6 )
    plt.plot(Tpv[mask], fcalc[i][mask]/max(fnew[i][mask]) +shift[k], '--', color='red', linewidth=3)
    plt.text(k*1, 5, str(percentage[i]))
    k=k+1

plt.xlabel('T (K)',fontsize=20)
plt.ylabel('Frequency (a.u., normalized)',fontsize=20)

plt.tick_params(axis="x",direction="in", labelsize=18, bottom='True', top='True', left='True', right='True', length=4, width=1, which = 'major')
plt.tick_params(axis="y",direction="in", labelsize=0, bottom='True', top='True', left='False', right='False', length=0, width=0, which = 'major')

plt.xlim(1.285, 1.655)

plt.legend(fontsize=18)

In [193]:
print(percentage[1],
percentage[5],
percentage[6],
percentage[8])

[ 0.98670544 -0.02501142  0.03830598] [ 0.61239861 -0.00215793  0.38975933] [ 1.05873344  0.17130866 -0.23004209] [0.51213693 0.24516273 0.24270034]
